# MLRun secret handling using Vault - Demo

This notebook demonstrates secret creation and handling in MLRun.


In [ ]:
from mlrun import new_task, get_run_db, new_function, run_local
import os
from mlrun.utils import add_vault_project_secret, add_vault_user_secret
from mlrun import mlconf, code_to_function, new_project

## Create a project
When a project is created, the `use_vault` parameter can be used to request that the underlying framework is created that will enable Vault secrets to be used with this project.
When selected, calling `.store()` on the project will save the project in the MLRun DB and the following will be created for the project (if not already existing):

1. A k8s serviceaccount 
2. A Vault policy that enables access to secrets in the project path (`/secrets/secret/projects/{proj name}`)
3. A Vault k8s role that associates the SA's token with the policy

These configurations are performed on the MLRun API server side, not from the client.

In [ ]:
proj_name = 'vault-mlrun'
func_name = 'vault-func'

proj = new_project(proj_name, use_vault=True)
proj.save(to_db=True, to_file=False)

The `create_vault_secrets` utility function allows creating project secrets, and will place them in the project secret path.
>Note: This method runs on client-side, which means the computer hosting this notebook must have Vault connectivity (and access Token) for this to work.

In [ ]:
proj.create_vault_secrets({'aws_key': '1234567890', 'github_key': 'proj1Key!!!'})
proj.get_vault_secret_keys()

## Create and deploy a function

In [ ]:
func_path = os.path.abspath("vault_function.py")

func = code_to_function(name=func_name,
                        filename=func_path,
                        handler='vault_func',
                        project=proj_name,
                        kind='job')

func.doc()

In [ ]:
func.spec.image = '.mlrun-vault-image'

In [ ]:
func.spec.build.base_image = 'saarcoiguazio/mlrun:unstable'
func.spec.build.image = '.mlrun-vault-image'
func.deploy()

In [ ]:
from mlrun import import_function

func = import_function(url='db://{}/{}'.format(proj_name,func_name))
func.doc()

## Initialize function runtime and execute
The `.with_secrets` function has a '`vault`' secret kind that will pass the specified Vault project secrets to the function context. The function spec
only contains the keys of the secrets ('aws_key' etc.) - the actual secret value is retrieved from Vault and planted in the function 
context in runtime.

In [ ]:
task = new_task(project=proj_name,
                name='vault_test_run',
                handler='vault_func',
                params={'secrets':['password', 'github_key', 'aws_key']})

# Add access to project-level secrets
# task.with_secrets('vault', ["aws_key", "github_key"])
task.with_secrets('vault', [])

# Can also use the following code to add user-level secrets
# task.with_secrets('vault', {"user": "admin", "secrets": ["password", "user_id"]})

print(task.to_yaml())

In [ ]:
result = func.run(task)

## Run the same function in another project's context
We will create a 2nd project, and assign different secret values to it. When the same function is executed in the new project's runtime context, it will get 
the new project's secrets. When running in this context, the function has no access to other projects' secrets.

In [ ]:
proj_name_2 = 'vault-mlrun-2'
proj2 = new_project(proj_name_2, use_vault=True)
proj2.save(to_db=True, to_file=False)
proj2.create_vault_secrets({'aws_key': '0987654321', 'github_key': 'proj2Key???', 'password': 'myPassword'})

In [ ]:
task2 = new_task(project=proj_name_2,
                 name='vault_test_run_2',
                 handler='vault_func',
                 params={'secrets':['password', 'github_key', 'aws_key']})
task2.with_secrets('vault', ["aws_key", "github_key", "password"])

result = func.run(task2)

In [ ]:
proj2.get_vault_secret_keys()